In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator
import multi_LLM
import multi_context
from bot import translate_text

In [ ]:
langs_dict = GoogleTranslator().get_supported_languages(as_dict=True)
print(langs_dict)

In [ ]:
langs_dict

In [ ]:
def eng_multi_LLM(question):
    response = multi_LLM.getResponse(question)
    english_response = response["answer"]
    relevant_chunks = response["relevant_chunks"]

    return english_response, relevant_chunks

def eng_multi_context(question):
    response = multi_context.getResponse(question)
    english_response = response["answer"]
    relevant_chunks = response["relevant_chunks"]

    return english_response, relevant_chunks

def translate_multi_LLM(question, dest_language):
    translated_question = translate_text(question, "en")
    response = multi_LLM.getResponse(translated_question)
    english_response = response["answer"]
    relevant_chunks = response["relevant_chunks"]
    translated_response = translate_text(english_response, dest_language)

    return english_response, translated_response, relevant_chunks

def translate_multi_context(question, dest_language):
    translated_question = translate_text(question, "en")
    response = multi_context.getResponse(translated_question)
    english_response = response["answer"]
    relevant_chunks = response["relevant_chunks"]
    translated_response = translate_text(english_response, dest_language)

    return english_response, translated_response, relevant_chunks

# Read from File & Defining Dest Language

In [ ]:
filepath = 'original'
translated = pd.read_csv(f"{filepath}.csv")

dest_language = 'zh-CN'

In [ ]:
translated[f"'question ({dest_language})"] = translated['question'].apply(lambda x: translate_text(x, dest_language))

# Multi-Context

In [ ]:
# ask question in english
translated[['multi_context answer', 'multi_context relevant chunks']] = translated['question'].apply(lambda x: pd.Series(eng_multi_context(x)))

translated[['multi_context answer (english translated)', f"multi_context answer ({dest_language})", f"multi_context relevant chunks ({dest_language})"]] = translated[f"'question ({dest_language})"].apply(lambda x: pd.Series(translate_multi_context(x, dest_language)))

# Multi LLM

In [ ]:
# ask question in english
translated[['multi_LLM answer', 'multi_LLM relevant chunks']] = translated['question'].apply(lambda x: pd.Series(eng_multi_LLM(x)))

translated[['multi_LLM answer (english translated)', f"multi_LLM answer ({dest_language})", f"multi_LLM relevant chunks ({dest_language})"]] = translated[f"'question ({dest_language})"].apply(lambda x: pd.Series(translate_multi_LLM(x, dest_language)))

# Saving File

In [ ]:
translated.to_csv(f"{filepath}_translated_{dest_language}.csv", index=False, encoding='utf-8-sig')